In [1]:
import cv2
import os
from keras.models import load_model
import numpy as np
from pygame import mixer
from PIL import Image, ImageTk
import time
import tkinter as tk
from tkinter import filedialog
from tkinter import messagebox
from tkinter import *
from tkinter import Label, Tk


pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
face = cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')
leye = cv2.CascadeClassifier('haarcascade_lefteye_2splits.xml')
reye = cv2.CascadeClassifier('haarcascade_righteye_2splits.xml')

In [3]:
lbl=['Close','Open']

In [4]:
model = load_model('eye_cnn.h5')
font = cv2.FONT_HERSHEY_COMPLEX_SMALL

In [ ]:
def open_webcam():
    cap = cv2.VideoCapture(0)
    rpred = [99]  # Initialize rpred
    lpred = [99]  # Initialize lpred
    while True:
        ret, frame = cap.read()
        height, width = frame.shape[:2]
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        faces = face.detectMultiScale(gray, minNeighbors=5, scaleFactor=1.1, minSize=(25, 25))
        left_eye = leye.detectMultiScale(gray)
        right_eye = reye.detectMultiScale(gray)
        cv2.rectangle(frame, (0, height - 50), (200, height), (0, 0, 0), thickness=cv2.FILLED)
        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (100, 100, 100), 1)
        for (x, y, w, h) in right_eye:
            r_eye = frame[y:y + h, x:x + w]
            r_eye = cv2.cvtColor(r_eye, cv2.COLOR_BGR2RGB)
            r_eye = cv2.resize(r_eye, (32, 32))
            r_eye = r_eye.reshape((-1, 32, 32, 3))
            rpred = np.argmax(model.predict(r_eye), axis=-1)
            if rpred[0] == 1:
                lbl = 'Open'
            elif rpred[0] == 0:
                lbl = 'Closed'
            break
        for (x, y, w, h) in left_eye:
            l_eye = frame[y:y + h, x:x + w]
            l_eye = cv2.cvtColor(l_eye, cv2.COLOR_BGR2RGB)
            l_eye = cv2.resize(l_eye, (32, 32))
            l_eye = l_eye.reshape((-1, 32, 32, 3))
            lpred = np.argmax(model.predict(l_eye), axis=-1)
            if lpred[0] == 1:
                lbl = 'Open'
            elif lpred[0] == 0:
                lbl = 'Closed'
            break
        if rpred[0] == 0 and lpred[0] == 0:
            cv2.putText(frame, "Closed", (10, height - 20), font, 1, (255, 255, 255), 1, cv2.LINE_AA)
        else:
            cv2.putText(frame, "Open", (10, height - 20), font, 1, (255, 255, 255), 1, cv2.LINE_AA)
        cv2.imshow('frame', frame)
        if cv2.waitKey(1) & 0xFF == ord('s'):
            break
    cap.release()
    cv2.destroyAllWindows()

def on_click_open_webcam():
    messagebox.showinfo("Opening Webcam", "Opening Webcam. Press 's' to close.")
    open_webcam()

# Creating GUI
root = tk.Tk()
root.title("Drowsiness Detection")
root.geometry('800x600')
root.configure(background='#CDCDCD')

heading = Label(root,text='Drowsiness Detection',pady=20,font=('arial',25,'bold'))
heading.configure(background='#CDCDCD',foreground="#364156")
heading.pack()

btn_open_webcam = tk.Button(root, text="Open Webcam", command=on_click_open_webcam)
btn_open_webcam.pack()

root.mainloop()


1/1 [==============================] - 0s 28ms/step


1/1 [==============================] - 0s 28ms/step
